Universidade de São Paulo - USP
Instituto de Matemática e Estatística - IME

Disciplina:  Ciência e Engenharia de Dado - MAC 5865
Responsável: Prof. Dr. Roberto Hirata
Estudantes: Alexandre Locci Martins, Angel Felipe M. de Paula

São Paulo, 16 de Novembro de 2018

Segundo Trabalho de Curso

Objetivos

Estudar o banco de dados "dvd_rental" com a finalidade de responder as seguintes perguntas:

1) O Banco de Dados é real ou Irreal? (feita pelo professor)

2) Quais os clientes que mais gastaram em locações? (feita pelo grupo)

3) Quantos clientes alugaram uma dada fita "X"? (feita pelo grupo)

4) Quais os atores que aparecem em mais filmes? (feita pelo professor) 

As respostas são apresentadas nas próximas células deste notebook.

Observação

Para realizar este trabalho, geramos um dataset (dataset_quatro.csv) que permite justificar as conclusões apresentadas, bem como armazena os resultados sobre os quais estas conclusões foram construídas.

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///dvd_rental.db

'Connected: @dvd_rental.db'

Respostas

Resposta questão 4 - Quais os atores que aparecem em mais filmes? (feita pelo professor) 
Para responder esta pergunta seguimos o seguinte procedimento:

1. Pelo diagrama de relacionamento identificamos que um ator pode estar em um filme e um filme pode ter um ator ou nenhum ator.
2. Implementamos os seguintes buscas (apresentadas nas celulas a seguir)

In [3]:
%sql ---Para determinar qual ator participou de qual filme implementamos a seguinte busca.

%sql Create table ator_1 as select film_actor.film_id, actor.actor_id from film_actor INNER JOIN actor ON film_actor.actor_id = actor.actor_id and actor.actor_id= 1;

%sql ---Criamos uma tabela ator_1 para armazenar os dados coletados. Nesta tabela, o ator com id = 1 é relacionado a cada
%sql ---filme no qual ele participou.

 * sqlite:///dvd_rental.db
0 rows affected.
 * sqlite:///dvd_rental.db
Done.
 * sqlite:///dvd_rental.db
0 rows affected.
 * sqlite:///dvd_rental.db
0 rows affected.


[]

In [4]:
%sql ---Agregamos a tabela ator_1 como uma coluna na tabela dataset_quatro para futura manipulação de dados.

%sql ALTER TABLE dataset_quatro ADD ator_1 int AFTER num_de;

%sql --- A coluna já existe, pois fizemos os testes antes entregar o trabalho.
%sql --- Logo, a saída desta célula vai resultar em erro, por duplicação de coluna.

 * sqlite:///dvd_rental.db
0 rows affected.
 * sqlite:///dvd_rental.db
(sqlite3.OperationalError) duplicate column name: ator_1 [SQL: 'ALTER TABLE dataset_quatro ADD ator_1 int AFTER num_de;'] (Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///dvd_rental.db
0 rows affected.
 * sqlite:///dvd_rental.db
0 rows affected.


[]

In [5]:
%sql ---Para todas as celulas na coluna ator_1 nos preenchemos com o valor zero. Depois executamos o comando abaixo.
%sql ---Neste comando, toda vez que film_id do dataset_quatro (que no banco é uma tabela) for igual ao film_id da tabela ator_1
%sql ---nós inserimos o valor 1.

%sql UPDATE dataset_quatro SET ator_1 = 1 WHERE film_id IN (SELECT film_id FROM ator_1 WHERE dataset_quatro.film_id = ator_1.film_id);

 * sqlite:///dvd_rental.db
0 rows affected.
 * sqlite:///dvd_rental.db
0 rows affected.
 * sqlite:///dvd_rental.db
0 rows affected.
 * sqlite:///dvd_rental.db
19 rows affected.


[]

In [6]:
%sql ---Para manter a integridade do banco, deletamos a tabela auxiliaer ator_1.

%sql DROP TABLE ator_1;

 * sqlite:///dvd_rental.db
0 rows affected.
 * sqlite:///dvd_rental.db
Done.


[]

In [7]:
%sql ---O processo acima descrito foi realizado para os 200 atores identificados no banco.
%sql ---Para automatizar o processo, e manter a simplicidade das buscas, foi criado um script utilizando um programa
%sql ---escrito em Java (arquivo entregue em anexo), o qual gerou todas os comandos sql necessários. O resultado, pode ser
%sql ---visto na tabela dataset_quatro.

 * sqlite:///dvd_rental.db
0 rows affected.
 * sqlite:///dvd_rental.db
0 rows affected.
 * sqlite:///dvd_rental.db
0 rows affected.
 * sqlite:///dvd_rental.db
0 rows affected.


[]

In [8]:
#%sql select * from dataset_quatro;
#Descomente para ver a tabela criada

Convertemos a tabela dataset_quatro em um panda DataFrame a fim de fazermos as devidas analises que vão determianr os atores que participaram de mais filmes.

In [9]:
#Criamos o dataframe partindo da tebela dataset_quatro
import sqlite3
import pandas as pd

con = sqlite3.connect('dvd_rental.db')
myFrames = pd.read_sql_query('select * from dataset_quatro;', con)

In [10]:
df = pd.DataFrame(myFrames)

In [11]:
#Selecionamos as colunas referentes aos atores.
df_aux = df.iloc[:, 9:209]
#Somamos o número de participação de cada ator.
df_aux.sum()
#Criamos um dataframe com base na soma dos valores 
df_ator = pd.DataFrame(df_aux.sum())
#Ordenamos o dataframe com base no número de participações.
ator = df_ator.sort_values(by=[0],ascending=False)
id_ator = ator.index

In [12]:
#Criamos a listagem.
for i in id_ator:
    valor = ator.loc[i].values[0]
    print("Participou de " + str(valor) + " filmes:")
    key = i.replace("ator_","") 
    first_name = pd.read_sql_query('select first_name from actor where actor_id = ' + key + ';', con)
    first_name = first_name['first_name'].values[0]
    last_name = pd.read_sql_query('select last_name from actor  where actor_id = ' + key + ';', con)
    last_name = last_name['last_name'].values[0]
    name = first_name + " " + last_name
    print(name)

Participou de 42 filmes:
GINA DEGENERES
Participou de 40 filmes:
MARY KEITEL
Participou de 39 filmes:
WALTER TORN
Participou de 37 filmes:
MATTHEW CARREY
Participou de 36 filmes:
SANDRA KILMER
Participou de 35 filmes:
VAL BOLGER
Participou de 35 filmes:
VIVIEN BASINGER
Participou de 35 filmes:
ANGELA WITHERSPOON
Participou de 35 filmes:
HENRY BERRY
Participou de 35 filmes:
SCARLETT DAMON
Participou de 34 filmes:
GROUCHO DUNST
Participou de 34 filmes:
ANGELA HUDSON
Participou de 33 filmes:
SEAN GUINESS
Participou de 33 filmes:
SIDNEY CROWE
Participou de 33 filmes:
SUSAN DAVIS
Participou de 33 filmes:
UMA WOOD
Participou de 33 filmes:
WARREN NOLTE
Participou de 33 filmes:
RENEE TRACY
Participou de 33 filmes:
EWAN GOODING
Participou de 33 filmes:
JAYNE NOLTE
Participou de 33 filmes:
RIP CRAWFORD
Participou de 32 filmes:
JULIA MCQUEEN
Participou de 32 filmes:
BEN WILLIS
Participou de 32 filmes:
KEVIN GARLAND
Participou de 32 filmes:
NATALIE HOPKINS
Participou de 32 filmes:
RENEE BALL
Parti

Conclusão: A lista da célula anterior identificva os atores e o número de filmes nos quais participou. A atriz Gina Degeneres participou de 42 filmes sendo a atriz com maior participação.

Resposta questão 3 - Quantos clientes alugaram uma dada fita "X"?

Pelo diagrama de relacionamento, um filme pode ter várias cópias no inventário, mas cada cópia possua um contrato de aluguel no qual um único cliente é identificado como um locatário. Logo, é possível determinar quantos clientes locaram um dado título, pois para cada vez que o inventtory_id da tabela inventory for igual ao inventory_id da tabela rental  que um filme, film_id, foi vinculado a um cliente, custumer_id.

In [13]:
%sql drop table film_customer;
%sql Create table film_customer as Select inventory.film_id, count(rental.customer_id) from rental INNER Join inventory where inventory.inventory_id = rental.inventory_id Group by inventory.film_id order by inventory.film_id;
#Tabela já foi criada durante os nossos testes.
%sql select * from film_customer;

 * sqlite:///dvd_rental.db
Done.
 * sqlite:///dvd_rental.db
Done.
 * sqlite:///dvd_rental.db
Done.


film_id,count(rental.customer_id)
1,23
2,7
3,12
4,23
5,12
6,21
7,15
8,18
9,12
10,23


Para melhorar a apresentação, criamos um dataframe para manipular os dados resultantes da pesquisa.

In [14]:
myFrames = pd.read_sql_query('Select * from film_customer;', con)
index = myFrames.index

In [15]:
for i in index:
    dado = str(myFrames.iloc[i,1])
    title = pd.read_sql_query('select title from film where film_id = ' + str(df.at[i,'film_id']) + ';', con)
    title = title.values[0]
    print("O filme " + title + " foi alugado por " + dado + " clientes.")

['O filme ACADEMY DINOSAUR foi alugado por 23 clientes.']
['O filme ACE GOLDFINGER foi alugado por 7 clientes.']
['O filme ADAPTATION HOLES foi alugado por 12 clientes.']
['O filme AFFAIR PREJUDICE foi alugado por 23 clientes.']
['O filme AFRICAN EGG foi alugado por 12 clientes.']
['O filme AGENT TRUMAN foi alugado por 21 clientes.']
['O filme AIRPLANE SIERRA foi alugado por 15 clientes.']
['O filme AIRPORT POLLOCK foi alugado por 18 clientes.']
['O filme ALABAMA DEVIL foi alugado por 12 clientes.']
['O filme ALADDIN CALENDAR foi alugado por 23 clientes.']
['O filme ALAMO VIDEOTAPE foi alugado por 24 clientes.']
['O filme ALASKA PHANTOM foi alugado por 26 clientes.']
['O filme ALI FOREVER foi alugado por 9 clientes.']
['O filme ALIEN CENTER foi alugado por 22 clientes.']
['O filme ALLEY EVOLUTION foi alugado por 14 clientes.']
['O filme ALONE TRIP foi alugado por 18 clientes.']
['O filme ALTER VICTORY foi alugado por 22 clientes.']
['O filme AMADEUS HOLY foi alugado por 21 clientes.']


['O filme COMA HEAD foi alugado por 26 clientes.']
['O filme COMANCHEROS ENEMY foi alugado por 6 clientes.']
['O filme COMFORTS RUSH foi alugado por 17 clientes.']
['O filme COMMAND DARLING foi alugado por 16 clientes.']
['O filme CONEHEADS SMOOCHY foi alugado por 25 clientes.']
['O filme CONFESSIONS MAGUIRE foi alugado por 14 clientes.']
['O filme CONFIDENTIAL INTERVIEW foi alugado por 29 clientes.']
['O filme CONFUSED CANDLES foi alugado por 17 clientes.']
['O filme CONGENIALITY QUEST foi alugado por 22 clientes.']
['O filme CONNECTICUT TRAMP foi alugado por 13 clientes.']
['O filme CONNECTION MICROCOSMOS foi alugado por 7 clientes.']
['O filme CONQUERER NUTS foi alugado por 16 clientes.']
['O filme CONSPIRACY SPIRIT foi alugado por 5 clientes.']
['O filme CONTACT ANONYMOUS foi alugado por 26 clientes.']
['O filme CONTROL ANTHEM foi alugado por 7 clientes.']
['O filme CONVERSATION DOWNHILL foi alugado por 17 clientes.']
['O filme CORE SUIT foi alugado por 18 clientes.']
['O filme COW

['O filme FLATLINERS KILLER foi alugado por 22 clientes.']
['O filme FLIGHT LIES foi alugado por 12 clientes.']
['O filme FLINTSTONES HAPPINESS foi alugado por 17 clientes.']
['O filme FLYING HOOK foi alugado por 20 clientes.']
['O filme FOOL MOCKINGBIRD foi alugado por 23 clientes.']
['O filme FOREVER CANDIDATE foi alugado por 13 clientes.']
['O filme FORREST SONS foi alugado por 18 clientes.']
['O filme FORRESTER COMANCHEROS foi alugado por 27 clientes.']
['O filme FORWARD TEMPLE foi alugado por 32 clientes.']
['O filme FREAKY POCUS foi alugado por 17 clientes.']
['O filme FREDDY STORM foi alugado por 23 clientes.']
['O filme FREEDOM CLEOPATRA foi alugado por 5 clientes.']
['O filme FRENCH HOLIDAY foi alugado por 17 clientes.']
['O filme FRIDA SLIPPER foi alugado por 11 clientes.']
['O filme FRISCO FORREST foi alugado por 9 clientes.']
['O filme FROGMEN BREAKING foi alugado por 10 clientes.']
['O filme FRONTIER CABIN foi alugado por 6 clientes.']
['O filme FROST HEAD foi alugado por 

['O filme JERICHO MULAN foi alugado por 17 clientes.']
['O filme JERK PAYCHECK foi alugado por 20 clientes.']
['O filme JERSEY SASSY foi alugado por 6 clientes.']
['O filme JET NEIGHBORS foi alugado por 20 clientes.']
['O filme JINGLE SAGEBRUSH foi alugado por 13 clientes.']
['O filme JOON NORTHWEST foi alugado por 17 clientes.']
['O filme JUGGLER HARDLY foi alugado por 32 clientes.']
['O filme JUMANJI BLADE foi alugado por 8 clientes.']
['O filme JUMPING WRATH foi alugado por 24 clientes.']
['O filme JUNGLE CLOSER foi alugado por 9 clientes.']
['O filme KANE EXORCIST foi alugado por 14 clientes.']
['O filme KARATE MOON foi alugado por 23 clientes.']
['O filme KICK SAVANNAH foi alugado por 20 clientes.']
['O filme KILLER INNOCENT foi alugado por 6 clientes.']
['O filme KING EVOLUTION foi alugado por 9 clientes.']
['O filme KISS GLORY foi alugado por 22 clientes.']
['O filme KISSING DOLLS foi alugado por 20 clientes.']
['O filme KNOCK WARLOCK foi alugado por 24 clientes.']
['O filme KRA

['O filme OTHERS SOUP foi alugado por 23 clientes.']
['O filme OUTBREAK DIVINE foi alugado por 23 clientes.']
['O filme OUTFIELD MASSACRE foi alugado por 20 clientes.']
['O filme OUTLAW HANKY foi alugado por 14 clientes.']
['O filme OZ LIAISONS foi alugado por 24 clientes.']
['O filme PACIFIC AMISTAD foi alugado por 24 clientes.']
['O filme PACKER MADIGAN foi alugado por 22 clientes.']
['O filme PAJAMA JAWBREAKER foi alugado por 14 clientes.']
['O filme PANIC CLUB foi alugado por 7 clientes.']
['O filme PANKY SUBMARINE foi alugado por 15 clientes.']
['O filme PANTHER REDS foi alugado por 15 clientes.']
['O filme PAPI NECKLACE foi alugado por 8 clientes.']
['O filme PARADISE SABRINA foi alugado por 18 clientes.']
['O filme PARIS WEEKEND foi alugado por 14 clientes.']
['O filme PARK CITIZEN foi alugado por 8 clientes.']
['O filme PARTY KNOCK foi alugado por 13 clientes.']
['O filme PAST SUICIDES foi alugado por 13 clientes.']
['O filme PATHS CONTROL foi alugado por 10 clientes.']
['O fil

['O filme VIRGIN DAISY foi alugado por 17 clientes.']
['O filme VIRGINIAN PLUTO foi alugado por 29 clientes.']
['O filme VIRTUAL SPOILERS foi alugado por 14 clientes.']
['O filme VISION TORQUE foi alugado por 7 clientes.']
['O filme VOICE PEACH foi alugado por 19 clientes.']
['O filme VOLCANO TEXAS foi alugado por 22 clientes.']
['O filme VOYAGE LEGALLY foi alugado por 28 clientes.']
['O filme WAGON JAWS foi alugado por 10 clientes.']
['O filme WAIT CIDER foi alugado por 21 clientes.']
['O filme WANDA CHAMBER foi alugado por 16 clientes.']
['O filme WAR NOTTING foi alugado por 12 clientes.']
['O filme WARDROBE PHANTOM foi alugado por 27 clientes.']
['O filme WARLOCK WEREWOLF foi alugado por 6 clientes.']
['O filme WARS PLUTO foi alugado por 10 clientes.']
['O filme WASH HEAVENLY foi alugado por 23 clientes.']
['O filme WASTELAND DIVINE foi alugado por 15 clientes.']
['O filme WATCH TRACY foi alugado por 16 clientes.']
['O filme WATERFRONT DELIVERANCE foi alugado por 17 clientes.']
['O 

Resposta Questão 1 - O Banco de Dados é real ou Irreal? (feita pelo professor)

Para provar que o banco não possui dados que expressão uma da realidade empiricamente verificável, ou seja, a locadora não existe, executamos a pesquisa abaixo.

In [16]:
%sql Select rental.staff_id, count(rental.rental_id) from rental group by rental.staff_id;

 * sqlite:///dvd_rental.db
Done.


staff_id,count(rental.rental_id)
1,8040
2,8004


Nesta pesquisa, notamos que 50,1% dos empréstimos foram feitos pela Loja 1 e 49,9% dos empréstimos foram feitos pela
Loja 2. Se o banco fosse construído com base em dados empíricos, deveria ser nota uma maior variação entre as porcentagens
 de aluguel, pois, em caso contrário, teríamos que admitir que variáveis que determinam qual loja aluga mais, assumem,
em última instância, os mesmo valores. Por outro lado, se o banco foi criado utilizando uma geração aleatória de dados,
na qual para duas variáveis L1 e L2 se tinha uma probabilidade de 50% de ser selecionada uma ou outra, teríamos, por
probabilidade frequentista um resultado similar ao constatado no banco para um número grande de sorteios. Logo,
acreditamos que este banco não possui dados reais pois há evidências de que os dados foram criados aleatoriamente.

Resposta Questão 2 - Quais os clientes que mais gastaram em locações? (feita pelo grupo)

Usando a tabela payment podemos verificar todos os pagamentos feitos por cada cliente. Com base nessa informação, agrupamos os clientes e somamos cada pagamento feito. O resultado é apresentado abaixo.

In [17]:
%sql Select payment.customer_id, customer.first_name, customer.last_name, sum(payment.amount) from payment INNER JOIN customer where payment.customer_id = customer.customer_id group by payment.customer_id order by sum(payment.amount) desc;

 * sqlite:///dvd_rental.db
Done.


customer_id,first_name,last_name,sum(payment.amount)
526,KARL,SEAL,221.5500000000001
148,ELEANOR,HUNT,216.5400000000001
144,CLARA,SHAW,195.58000000000007
137,RHONDA,KENNEDY,194.61000000000007
178,MARION,SNYDER,194.61000000000007
459,TOMMY,COLLAZO,186.62000000000006
469,WESLEY,BULL,177.60000000000002
468,TIM,CARY,175.61000000000004
236,MARCIA,DEAN,175.58
181,ANA,BRADLEY,174.66000000000005
